In [36]:
from bs4 import BeautifulSoup
import os
import requests
from pymongo import MongoClient
from pymongo.errors import DuplicateKeyError, CollectionInvalid
import datetime as dt
import unicodedata
import pandas as pd



In [16]:
def find_class(soup, class_string):
    '''
    helper function for getting class information out of the soup for a page
    INPUT:
        - soup, soup object for a page
        - class_string, string of class label to grab
    OUTPUT: soup result set
    '''
    return soup.findAll(class_=class_string)

def find_parks_list(soup):
    return soup.findAll(id_="list_parks")

# Replace the ca with all the state codes
nps_state_parks_url = 'https://www.nps.gov/state/ca/index.htm'
soup = BeautifulSoup(requests.get(nps_state_parks_url).content, 'html.parser')

'''
# Each park items
<div class="col-md-9 col-sm-9 col-xs-12 table-cell list_left">
<h2></h2># This field has the type. Possible values: National Monument,National Park,National Historic Site, National Recreation Area
<h3><a href="/alca/" id="anch_9">Alcatraz Island</a></h3> #  This has the name of the national site
<h4>San Francisco, CA</h4> # This field has City followed by , and then state code
<p>
Alcatraz Island offers a close-up look at the site of the first lighthouse and US built fort on the West Coast, the infamous federal penitentiary long off-limits to the public, and the history making 18 month occupation by Indians of All Tribes. Rich in history, there is also a natural side to the Rock—gardens, tide pools, bird colonies, and bay views beyond compare.
</p> # this field has the description
</div>

'''

parks_list = find_class(soup, "col-md-9 col-sm-9 col-xs-12 table-cell list_left")
parks_list


[<div class="col-md-9 col-sm-9 col-xs-12 table-cell list_left">
 <h2></h2>
 <h3><a href="/alca/">Alcatraz Island</a></h3>
 <h4>San Francisco, CA</h4>
 <p>
 Alcatraz Island offers a close-up look at the site of the first lighthouse and US built fort on the West Coast, the infamous federal penitentiary long off-limits to the public, and the history making 18 month occupation by Indians of All Tribes. Rich in history, there is also a natural side to the Rock—gardens, tide pools, bird colonies, and bay views beyond compare.
 </p>
 </div>, <div class="col-md-9 col-sm-9 col-xs-12 table-cell list_left">
 <h2>National Monument</h2>
 <h3><a href="/cabr/">Cabrillo</a></h3>
 <h4>San Diego, CA</h4>
 <p>
 Climbing out of his boat and onto shore in 1542, Juan Rodriguez Cabrillo stepped into history as the first European to set foot on what is now the West Coast of the United States. In addition to telling the story of 16th century exploration, the park is home to a wealth of cultural and natural res

In [5]:
# Save to mongodb the information from the web.
# Store both the whole url and the div components

db_client = MongoClient()
database = db_client['nationalparks']
collection = database['htmldata']

'''

db.post.insert([
   {
      title: 'MongoDB Overview', 
      description: 'MongoDB is no sql database',
      by: 'tutorials point',
      url: 'http://www.tutorialspoint.com',
      tags: ['mongodb', 'database', 'NoSQL'],
      likes: 100
   }
'''



"\n\ndb.post.insert([\n   {\n      title: 'MongoDB Overview', \n      description: 'MongoDB is no sql database',\n      by: 'tutorials point',\n      url: 'http://www.tutorialspoint.com',\n      tags: ['mongodb', 'database', 'NoSQL'],\n      likes: 100\n   }\n"

In [ ]:
collection.insert_one({"url" : "https://www.nps.gov/state/ca/index.htm", "raw_html" : soup.prettify() })


In [12]:
h = collection.insert_one({"url" : "https://www.nps.gov/state/ca/index.htm", "raw_html" : soup.prettify() })
h

In [6]:
res = collection.find()
res.next()


{'_id': ObjectId('5c6255cde80eff1d9363e428'),
 'url': 'https://www.nps.gov/state/ca/index.htm',
 'raw_html': '<!DOCTYPE doctype html>\n<html class="no-js" lang="en">\n <!-- Content Copyright National Park Service -->\n <!-- JavaScript & DHTML Code Copyright &copy; 1998-2017, PaperThin, Inc. All Rights Reserved. -->\n <head>\n  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>\n  <title>\n   California (U.S. National Park Service)\n  </title>\n  <script>\n   var jsDlgLoader = \'/state/ca/loader.cfm\';\nvar jsSiteResourceLoader = \'/cs-resources.cfm?r=\';\nvar jsSiteResourceSettings = {canCombine: false, canMinify: false};\n  </script>\n  <!-- beg (1) PrimaryResources -->\n  <script src="/ADF/thirdParty/jquery/jquery-1.12.js" type="text/javascript">\n  </script>\n  <script src="/commonspot/javascript/browser-all.js" type="text/javascript">\n  </script>\n  <!-- end (1) PrimaryResources -->\n  <!-- beg (2) SecondaryResources -->\n  <!-- end (2) SecondaryResources -->\n  <!-- beg (3) S

In [13]:
# insert the parsed html
ca_parks_list = list(parks_list)
#h = collection.insert_one({"stateCode" : "CA", "park_id": 1, "parks_list" : ca_parks_list[0] })

hh = ca_parks_list[0].html
print(hh)

None


In [25]:

for idx, val in enumerate(parks_list):
    collection.insert_one({"stateCode" : "CA", "park_id": idx, "park_info" : unicodedata.normalize('NFKC', val.decode()).encode('ascii','ignore') })
    


In [48]:
def get_information(park, state_code):
    '''
    This will take the div tag of the park and return a key value pairs for the items I want to store.
      h2 tag had the type of the national site. Eg: National Monument,National Park,National Historic Site, National Recreation Area
     h3 tag had the name of the national site
     h4 tag had the city and state code separated by ,
      p tag had the description of the park.
    
     parkname:name of the park
     state:state where the park is located
     city:city of the park location
     type:type of park. Possible values:National Monument,National Park,National Historic Site, National Recreation Area
     description:text description of the park
    
    
    <div class="col-md-9 col-sm-9 col-xs-12 table-cell list_left">
 <h2>National Monument</h2>
 <h3><a href="/labe/">Lava Beds</a></h3>
 <h4>Tulelake, CA</h4>
 <p>
 Lava Beds National Monument is a land of turmoil, both geological and historical. Over the last half-million years, volcanic eruptions on the Medicine Lake shield volcano have created a rugged landscape dotted with diverse volcanic features. More than 700 caves, Native American rock art sites, historic battlefields and campsites, and a high desert wilderness experience await you!
 </p>
 </div>
    '''
    
    park_type = (park.find('h2')).contents
    
    if ((park.find('h2')).contents):
        park_type = (park.find('h2')).contents[0]
    else:
        park_type = ""
    park_name = ((park.find('h3')).find('a')).contents[0]
    
    # Park city was throwing an error because of the missing city so putting a check
    if((park.find('h4')).contents):
        park_city = ((park.find('h4')).contents[0]).split(",")[0]
    else:
        park_city = ""
    park_state = state_code
    park_description = (park.find('p')).contents[0]
    
    '''
    print(park_type)
    print(park_name)
    print(park_city)
    print(park_state)
    print(park_description)
    '''
    
    #create a dictionary so you can insert the values into mongo
    park_dict = {"park_name": park_name, "park_type":park_type, "park_city": park_city, "park_state": park_state, "park_desc": park_description}
    return park_dict
    

    
    
park_dict = get_information(parks_list[0],'CA')
print(park_dict)

# insert into mongo the park_dict 
#collection.insert_one(park_dict)



    
    
    

{'park_name': 'Alcatraz Island', 'park_type': '', 'park_city': 'San Francisco', 'park_state': 'CA', 'park_desc': '\nAlcatraz Island offers a close-up look at the site of the first lighthouse and US built fort on the West Coast, the infamous federal penitentiary long off-limits to the public, and the history making 18 month occupation by Indians of All Tribes. Rich in history, there is also a natural side to the Rock—gardens, tide pools, bird colonies, and bay views beyond compare.\n'}


In [41]:
state_info = pd.read_csv("../data/state_province_codes.csv",skiprows=1)
state_list = state_info['State_Cd'].tolist()
state_list



['AL',
 'AK',
 'AZ',
 'AR',
 'CA',
 'CO',
 'CT',
 'DE',
 'DC',
 'FL',
 'GA',
 'HI',
 'ID',
 'IL',
 'IN',
 'IA',
 'KS',
 'KY',
 'LA',
 'ME',
 'MD',
 'MA',
 'MI',
 'MN',
 'MS',
 'MO',
 'MT',
 'NE',
 'NV',
 'NH',
 'NJ',
 'NM',
 'NY',
 'NC',
 'ND',
 'OH',
 'OK',
 'OR',
 'PA',
 'RI',
 'SC',
 'SD',
 'TN',
 'TX',
 'UT',
 'VT',
 'VA',
 'WA',
 'WV',
 'WI',
 'WY',
 'AS',
 'GU',
 'MH',
 'FM',
 'MP',
 'PW',
 'PR',
 'VI']

In [50]:

def pipeline_to_insert_data(state_codes_list):
    nps_parks_data_to_insert_list = []

    for i in range(0,len(state_codes_list)):
        url = 'https://www.nps.gov/state/%s/index.htm' % (state_codes_list[i].lower())
        soup = BeautifulSoup(requests.get(url).content, 'html.parser')
        # Construct entries to insert the raw html
        insert_html_row = {"state_cd":state_codes_list[i],\
                      "url":url,\
                      "raw_html":soup.prettify()}
        
        nps_parks_data_to_insert_list.append(insert_html_row)
        # Get the park list as a soup resultlist object from the soup object
        parks_list = find_class(soup, "col-md-9 col-sm-9 col-xs-12 table-cell list_left")
        for j in range(0,len(parks_list)):
            park_information_row = get_information(parks_list[j],state_codes_list[i])
            nps_parks_data_to_insert_list.append(park_information_row)

        
        
    return nps_parks_data_to_insert_list


def get_mongo_collection():
    db_client = MongoClient()
    database = db_client['nationalparks']
    collection = database['htmldata']
    return collection

#Construct the data 
data_to_insert = pipeline_to_insert_data(state_list)

#Get the Mongo Collection
#collection = get_mongo_collection()
#collection.insert_many(data_to_insert)
        
#nps_state_parks_url = 'https://www.nps.gov/state/ca/index.htm'

data_to_insert

# soup = BeautifulSoup(requests.get(nps_state_parks_url).content, 'html.parser')

[{'state_cd': 'AL',
  'url': 'https://www.nps.gov/state/al/index.htm',
  'raw_html': '<!DOCTYPE doctype html>\n<html class="no-js" lang="en">\n <!-- Content Copyright National Park Service -->\n <!-- JavaScript & DHTML Code Copyright &copy; 1998-2017, PaperThin, Inc. All Rights Reserved. -->\n <head>\n  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>\n  <title>\n   Alabama (U.S. National Park Service)\n  </title>\n  <script>\n   var jsDlgLoader = \'/state/al/loader.cfm\';\nvar jsSiteResourceLoader = \'/cs-resources.cfm?r=\';\nvar jsSiteResourceSettings = {canCombine: false, canMinify: false};\n  </script>\n  <!-- beg (1) PrimaryResources -->\n  <script src="/ADF/thirdParty/jquery/jquery-1.12.js" type="text/javascript">\n  </script>\n  <script src="/commonspot/javascript/browser-all.js" type="text/javascript">\n  </script>\n  <!-- end (1) PrimaryResources -->\n  <!-- beg (2) SecondaryResources -->\n  <!-- end (2) SecondaryResources -->\n  <!-- beg (3) StyleTags -->\n  <!-- end (3